In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.legacy as legacy
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchwordemb

In [101]:
ING_WORD2VEC_DIM = 300
ING_RNN_DIM = 300
ING_WORD2VEC_PATH = '/home/yifu/Documents/Mycode/python/hierarchicalRNN/jasha/files/vocab.bin'
IMAGE_MODEL = 'resNet50'
IMAGE_LAYER_FINAL = 2048
EMBD_DIM = 1024

In [102]:
class ingredient_RNN (nn.Module):
    def __int__(self):
        super(ingredient_RNN, self).__init__()
        
        self.irnn = nn.LSTM(input_size= ING_WORD2VEC_DIM, hidden_size=ING_RNN_DIM, bidirectional=True, batch_first=True)
        
        #Get the size of the Vocab.
        _, vec = torchwordemb.load_word2vec_bin(ING_WORD2VEC_PATH) # give the vector of size 300
        
        #Creating the Embedding Matrix and then copy the vectors from Google WORD2VEC model to the embedding variable
        self.embs = nn.Embedding(vec.size(0), ING_WORD2VEC_DIM, padding_idx=0) # not sure about the padding idx
        
        self.embs.weight.data.copy_(vec)
    
    def forward(self, x, seq_lengths):
        # X is the variable, seq_lengths is the length of the ingredients of single Recipe
        
        ################################
        # Following things need to be under stood properly.
        #################################
        # sort sequence according to the length
        sorted_len, sorted_idx = sq_lengths.sort(0, descending=True)
        
        index_sorted_idx = sorted_idx\
                            .view(-1,1,1).expand_as(x)
        sorted_inputs = x.gather(0, index_sorted_idx.long())
        
        # pack sequence
        packed_seq = torch.nn.utils.rnn.pack_padded_sequence(
                sorted_inputs, sorted_len.cpu().data.numpy(), batch_first=True)
        
        # pass it to the rnn
        out, hidden = self.irnn(packed_seq)

        # unsort the output
        _, original_idx = sorted_idx.sort(0, descending=False)

        # LSTM
        # bi-directional
        unsorted_idx = original_idx.view(1,-1,1).expand_as(hidden[0])
        
        # 2 directions x batch_size x num features, we transpose 1st and 2nd dimension
        
        output = hidden[0].gather(1,unsorted_idx).transpose(0,1).contiguous()
        output = output.view(output.size(0),output.size(1)*output.size(2))

        return output

In [5]:
embs = nn.Embedding(30566, ING_WORD2VEC_DIM, padding_idx=0)
a =  torch.tensor([[[1,2], [2,3],[3,4]],[[1,2], [2,3],[3,4]]])
b = [1,23,34,23,23,34,34,5446,56,5,34,0]
c = torch.LongTensor(b)
import numpy as np
d = max(np.nonzero(b)[0]) +1
e = torch.autograd.Variable(c)
x = embs(e)
#d\.sort(0, descending=True)

TypeError: 'module' object is not callable

In [6]:
# Following thing need to be understood completely 
def norm(input, p=2, dim=1, eps=1e-12):
    return input / input.norm(p,dim,keepdim=True).clamp(min=eps).expand_as(input)

In [7]:
class image_ingredient(nn.Module):
    def __init__(self):
        super(image_ingedient, self).__init__()
        if IMAGE_MODEL =='resNet50':

            resnet = models.resnet50(pretrained=True)
            modules = list(resnet.children())[:-1]  # we do not use the last fc layer.
            self.visionMLP = nn.Sequential(*modules)

            self.visual_embedding = nn.Sequential(
                nn.Linear(IMAGE_LAYER_FINAL, EMBD_DIM),
                nn.Tanh(),
            )

            self.ingredient_embedding = nn.Sequential(
                #ING_RNN_DIM is doubtful here.  If it needs to be multiplied or not.
                nn.Linear(ING_RNN_DIM*3, EMBD_DIM, EMBD_DIM),
                nn.Tanh(),
            )

        else:
            raise Exception('Only resNet50 model is implemented.')

        self.ingRNN_    = ingredient_RNN()

    def forward(self, x, z1, z2): # we need to check how the input is going to be provided to the model
        
        #############################
        # X is the image matrix.
        # Change the input according, z1 and z2 retains the last two parameters for the ing_vec and sequence length
        ####################################
       
        # Ingredient embedding
        ingredient_emb = self.ingRNN_(z1,z2)
        ingredient_emb = self.ingredient_embedding(ingredient_emb)
        
        ingredient_emb = norm(ingredient_emb)

        # visual embedding
        visual_emb = self.visionMLP(x)
        visual_emb = visual_emb.view(visual_emb.size(0), -1)
        visual_emb = self.visual_embedding(visual_emb)
        visual_emb = norm(visual_emb)

        output = [visual_emb, ingredient_emb]
        return output


In [98]:
x = (1 + torch.rand(160) * (1000 - 1)).type(torch.int16)

In [99]:
x = s

tensor([ 506,  518,  367,  999,  547,  322,  219,  951,  702,   60,
         554,  343,  556,  103,  906,  612,   68,  673,  686,   54,
         562,  255,  963,  486,  365,  156,   99,  637,  903,  223,
         473,  271,  809,  846,  324,  678,  464,  969,   37,  440,
         563,  177,   53,  858,  900,  833,  567,  396,  516,  561,
         228,  201,  790,   28,  330,  164,  122,  433,   74,  996,
         166,  302,  973,  441,  987,  414,  621,  102,  392,  415,
         450,  558,  621,  645,  100,  847,  400,  640,  279,  564,
         352,  987,  427,  475,  542,  481,  956,  801,  711,  907,
         128,  989,  888,  157,  190,  877,  577,  422,  810,  514,
         779,  128,  940,  227,  189,  951,  475,  738,  100,  937,
         773,  804,  704,   64,  628,  428,  468,  324,  803,  468,
         690,  829,  763,  337,  349,  671,  668,   91,  985,  250,
         407,  954,  288,  855,   83,  472,  785,  662,  350,  481,
         823,   58,  341,  666,  438,  196,  344

In [104]:
s = 1 + torch.rand(160) * (20 - 1)


NameError: name 'self' is not defined

In [58]:
s_l,s_idx = s.sort(0,descending=True)

In [63]:
s_l.type(torch.int16)

tensor([ 19,  19,  19,  19,  19,  19,  19,  19,  19,  18,  18,  18,
         17,  17,  17,  17,  17,  17,  17,  17,  17,  17,  16,  16,
         16,  16,  16,  16,  16,  16,  16,  16,  16,  15,  15,  15,
         15,  15,  15,  15,  14,  14,  14,  14,  14,  14,  13,  13,
         13,  13,  13,  12,  12,  12,  12,  12,  12,  12,  11,  11,
         11,  11,  11,  11,  11,  11,  10,  10,  10,  10,  10,  10,
         10,  10,  10,  10,  10,  10,  10,   9,   9,   8,   8,   8,
          8,   8,   8,   8,   8,   7,   7,   7,   7,   7,   7,   7,
          7,   7,   7,   6,   6,   6,   6,   6,   6,   6,   6,   5,
          5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,   5,
          4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,   4,
          4,   3,   3,   3,   3,   2,   2,   2,   2,   2,   2,   2,
          1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
          1,   1,   1,   1], dtype=torch.int16)

In [79]:
y = s_idx.view(-1,1,1)#.expand_as(x.unsqueeze(2))

In [80]:
y

tensor([[[  71]],

        [[  13]],

        [[  47]],

        [[  49]],

        [[  94]],

        [[  69]],

        [[ 142]],

        [[ 109]],

        [[  65]],

        [[  89]],

        [[ 125]],

        [[ 138]],

        [[  43]],

        [[ 101]],

        [[  95]],

        [[ 120]],

        [[  18]],

        [[  72]],

        [[  21]],

        [[   8]],

        [[  51]],

        [[  30]],

        [[   0]],

        [[  35]],

        [[  11]],

        [[  93]],

        [[  87]],

        [[   5]],

        [[  70]],

        [[  10]],

        [[ 156]],

        [[ 103]],

        [[ 130]],

        [[  84]],

        [[ 149]],

        [[  42]],

        [[  74]],

        [[ 110]],

        [[  14]],

        [[   1]],

        [[  16]],

        [[  40]],

        [[ 140]],

        [[ 119]],

        [[  81]],

        [[  92]],

        [[ 123]],

        [[  75]],

        [[  26]],

        [[  34]],

        [[   7]],

        [[ 129]],

        [[ 1

In [73]:
torch.nn.utils.rnn.pack_padded_sequence(s_i, s_l.cpu().data.numpy(), batch_first=True)

TypeError: slice indices must be integers or None or have an __index__ method

In [90]:
batch_size = 3
max_length = 3
hidden_size = 2
n_layers =1

In [91]:
# container
batch_in = torch.zeros((batch_size, 1, max_length))

In [92]:
batch_in

tensor([[[ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.]]])

In [93]:
#data
vec_1 = torch.FloatTensor([[1, 2, 3]])
vec_2 = torch.FloatTensor([[1, 2, 0]])
vec_3 = torch.FloatTensor([[1, 0, 0]])

In [94]:
vec_1

tensor([[ 1.,  2.,  3.]])

In [95]:
batch_in[0] = vec_1
batch_in[1] = vec_2
batch_in[2] = vec_3

In [96]:
from torch.autograd import Variable
batch_in = Variable(batch_in)

seq_lengths = [3,2,1] # list of integers holding information about the batch size at each sequence step


In [97]:
# pack it
pack = torch.nn.utils.rnn.pack_padded_sequence(batch_in, seq_lengths, batch_first=True)

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)